In [ ]:
# Importing libraries - pip install interpret if needed

In [1]:
import interpret

In [2]:
import pandas as pd
import numpy as np

In [3]:
from sklearn.model_selection import train_test_split

In [ ]:
# Prepare dataset

In [4]:
accepted = pd.read_csv('/Users/alexlei/Downloads/accepted_2007_to_2018Q4.csv', index_col=0, low_memory=False)

In [5]:
pd.options.display.max_rows = 200

In [ ]:
#accepted.columns.unique()

In [6]:
accepted.dtypes

member_id                                     float64
loan_amnt                                     float64
funded_amnt                                   float64
funded_amnt_inv                               float64
term                                           object
int_rate                                      float64
installment                                   float64
grade                                          object
sub_grade                                      object
emp_title                                      object
emp_length                                     object
home_ownership                                 object
annual_inc                                    float64
verification_status                            object
issue_d                                        object
loan_status                                    object
pymnt_plan                                     object
url                                            object
desc                        

In [ ]:
#accepted = accepted.dropna()

In [12]:
accepted.columns

Index(['member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term',
       'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       ...
       'hardship_payoff_balance_amount', 'hardship_last_payment_amount',
       'disbursement_method', 'debt_settlement_flag',
       'debt_settlement_flag_date', 'settlement_status', 'settlement_date',
       'settlement_amount', 'settlement_percentage', 'settlement_term'],
      dtype='object', length=150)

In [ ]:
# Perform train, test, split

In [14]:
X = accepted.iloc[:, :-1]

In [16]:
y = (accepted.iloc[:, -1] == " >50K").astype(int)

In [17]:
seed = 42
np.random.seed(seed)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)

In [ ]:
# Train a Glassbox Model

In [19]:
from interpret.glassbox import ExplainableBoostingClassifier
ebm = ExplainableBoostingClassifier()
ebm.fit(X_train, y_train)

/Users/alexlei/anaconda3/lib/python3.11/site-packages/interpret/utils/_preprocessor.py:314: RuntimeWarning: All-NaN slice encountered
  min_feature_val = np.nanmin(X_col)
/Users/alexlei/anaconda3/lib/python3.11/site-packages/interpret/utils/_preprocessor.py:315: RuntimeWarning: All-NaN slice encountered
  max_feature_val = np.nanmax(X_col)
/Users/alexlei/anaconda3/lib/python3.11/site-packages/interpret/glassbox/_ebm/_ebm.py:669: UserWarning: Missing values detected. Our visualizations do not currently display missing values. To retain the glassbox nature of the model you need to either set the missing values to an extreme value like -1000 that will be visible on the graphs, or manually examine the missing value score in ebm.term_scores_[term_index][0]
  warn(
/Users/alexlei/anaconda3/lib/python3.11/site-packages/interpret/glassbox/_ebm/_ebm.py:810: UserWarning: Only 1 class detected for classification. The model will predict 1.0 whenever predict_proba is called.
  warn(


ExplainableBoostingClassifier()

In [ ]:
# Explain the Glassbox - identify what the model finds important

In [20]:
from interpret import set_visualize_provider
from interpret.provider import InlineProvider
set_visualize_provider(InlineProvider())

In [21]:
from interpret import show
show(ebm.explain_global())

KeyboardInterrupt: 

In [ ]:
# Focus on Loan_Status column as the goal is to predict loan status

In [ ]:
accepted['loan_status'].unique()

In [ ]:
# 33 Null valuses in the column

In [ ]:
accepted['loan_status'].value_counts(dropna=False)

In [ ]:
# Replace all values to 2 categories - 'Paid', 'Late', or 'Default'

In [ ]:
replace_status = {"Fully Paid":"Paid",
                  "Current": "Paid",
                  "Charged Off": "Default",
                  "Late (31-120 days)":"Default",
                  "Late (16-30 days)":"Default",
                  "Does not meet the credit policy. Status:Charged Off":"Default",
                  "Does not meet the credit policy. Status:Fully Paid":"Paid",
                  "In Grace Period":"Default",
                  "Default":"Default"
                 }

In [ ]:
accepted['loan_status'] = accepted['loan_status'].replace(replace_status)

In [ ]:
accepted['loan_status'].value_counts(dropna=True)

In [ ]:
# Add a list of features

In [ ]:
features = accepted[['total_pymnt', 'annual_inc', 'fico_range_low', 'fico_range_high', 'total_acc', 'delinq_amnt', 'earliest_cr_line', 'emp_length', 'home_ownership', 'loan_amnt', 'term', 'pub_rec_bankruptcies', 'pub_rec', 'mort_acc']]

In [ ]:
status = accepted['loan_status']

In [ ]:
# Perform train, test, split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, status)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# Creating an instance of Logistic Regression with sag solver
model = LogisticRegression(solver='sag', max_iter=1000)

In [ ]:
# Fitting the model to the training data
model.fit(X_train, y_train)

In [ ]:




# Fitting the model to the training data
model.fit(X_train, y_train)

# Predicting labels for the test set
y_pred = model.predict(X_test)

# Evaluating the model performance
accuracy = model.score(X_test, y_test)
print("Accuracy:", accuracy)